In [1]:
import pandas as pd

In [2]:
data=pd.read_csv(r'C:\Users\My Pc\ML_project\text\text_summar\Topic.csv')
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [3]:
data.shape

(327337, 2)

In [4]:
data_text=data[:300000][['headline_text']]
data_text['index']=data_text.index

In [5]:
documents=data_text

In [6]:
#get the total no of douments

In [7]:
print(len(documents))

300000


In [8]:
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [9]:
# Data preprocessing.
#1.toeknization , stowpords, lemmatization, stemmed

In [10]:
#loading gensim and nltk libraries

In [11]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import*

C:\Users\My Pc\Anaconda3\Anaconda\envs\tensorflow\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [12]:
import numpy as np
np.random.seed(400)

In [13]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\My
[nltk_data]     Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
#before preprocesiing our datset, lets first look at an lemmatizing example

In [15]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


In [16]:
#throw a number of words at the stemmer and see how it deals with each one

In [17]:
stemmer=SnowballStemmer('english')
original_words=['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles=[stemmer.stem(plural) for plural in original_words]

In [18]:
pd.DataFrame(data={'original word': original_words, 'stemmed':singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [19]:
#performing the pre processing steps on the entire dataset

In [20]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer(). lemmatize(text, pos='v'))
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
    return result

In [21]:
#preview a document after preprocessing

In [22]:
document_num=4310
doc_sample=documents[documents['index']==document_num].values[0][0]
print('original document')
words=[]
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print(preprocess(doc_sample))

original document
['rain', 'helps', 'dampen', 'bushfires']
['rain', 'help', 'dampen', 'bushfir']


In [23]:
documents

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
...,...,...
299995,broughton hall audit reveals serious breaches,299995
299996,broughton hall fails key standards,299996
299997,broughton hall safe for residents govt says,299997
299998,burn off at conservation park aims to prevent,299998


In [24]:
#processing all the headlines , saving the list of results as 'processed_docs'
processed_docs=documents['headline_text'].map(preprocess)

In [25]:
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [26]:
#bag of words on the dataset
#create a dictionary from precessed_docs containing the number of times a word appers in the training set.

In [27]:
dictionary=gensim.corpora.Dictionary(processed_docs)

In [28]:
#checking dictionary created
count=0
for k,v in dictionary.iteritems():
    print(k,v)
    count+=1
    if count>10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [29]:
#remove very rare and common words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

In [30]:
#convert document into bag-of-words format. each word is assumed to tokenize and normalized string
bow_corpus=[dictionary.doc2bow(doc) for doc in processed_docs]

In [31]:
#checking bag of words for ur sample dcument
bow_corpus[document_num]

[(76, 1), (113, 1), (482, 1), (4016, 1)]

In [32]:
#preview bow for our sample preprocessed document
bow_doc_4310=bow_corpus[document_num]
for i in range(len(bow_doc_4310)):
    print('word {}(\'{}\') appears {} time'.format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

word 76('bushfir') appears 1 time
word 113('help') appears 1 time
word 482('rain') appears 1 time
word 4016('dampen') appears 1 time


In [33]:
#TF-IDF on our document set
#while perforing Tf-idf on the corpus is not necessary for LDA implementation using gensim model

In [34]:
#create tf-idf model object using modles
from gensim import corpora, models
tfidf=models.TfidfModel(bow_corpus)


In [35]:
#apply tranformation to the entire corpus and call it 'corpus-tfidf'
corpus_tfidf=tfidf[bow_corpus]

In [36]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


In [37]:
#running LDA using Bag of words
#train the lda mdel using gensim.models.LdaMulticore 
lda_model=gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2)

In [38]:
#fr each topic we will explore the words occuring in that topic and its relative weight
for idx, topic in lda_model.print_topics(-1):
    print('topic {} \nwords:{}'.format(topic,idx))
    print('\n')

topic 0.035*"polic" + 0.019*"opposit" + 0.018*"work" + 0.017*"call" + 0.017*"probe" + 0.016*"talk" + 0.016*"govt" + 0.013*"blaze" + 0.010*"threat" + 0.010*"push" 
words:0


topic 0.022*"minist" + 0.019*"closer" + 0.018*"claim" + 0.017*"protest" + 0.013*"strike" + 0.012*"reject" + 0.011*"australia" + 0.010*"worker" + 0.010*"damag" + 0.009*"storm" 
words:1


topic 0.023*"hospit" + 0.017*"report" + 0.016*"deal" + 0.013*"action" + 0.011*"find" + 0.010*"highlight" + 0.010*"inquiri" + 0.009*"bird" + 0.009*"rate" + 0.008*"studi" 
words:2


topic 0.038*"govt" + 0.022*"fund" + 0.020*"urg" + 0.016*"boost" + 0.012*"water" + 0.012*"farmer" + 0.011*"price" + 0.011*"group" + 0.011*"drought" + 0.010*"help" 
words:3


topic 0.023*"nation" + 0.018*"win" + 0.016*"rule" + 0.010*"final" + 0.010*"blue" + 0.010*"clash" + 0.009*"chang" + 0.009*"critic" + 0.009*"meet" + 0.009*"race" 
words:4


topic 0.024*"polic" + 0.019*"jail" + 0.016*"arrest" + 0.014*"break" + 0.012*"land" + 0.011*"famili" + 0.010*"suspect"

In [43]:
#performance evalution by classifying sample documents using LDA Bag of words
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [44]:
#check which topic our test document belongs to using LDA Bag of words model
document_num=4310
#our test document in document number
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print('\nscore:{}\t \ntopic {}'.format(score, lda_model.print_topic(index,10)))


score:0.4705457389354706	 
topic 0.038*"govt" + 0.022*"fund" + 0.020*"urg" + 0.016*"boost" + 0.012*"water" + 0.012*"farmer" + 0.011*"price" + 0.011*"group" + 0.011*"drought" + 0.010*"help"

score:0.3694206178188324	 
topic 0.023*"hospit" + 0.017*"report" + 0.016*"deal" + 0.013*"action" + 0.011*"find" + 0.010*"highlight" + 0.010*"inquiri" + 0.009*"bird" + 0.009*"rate" + 0.008*"studi"

score:0.020006079226732254	 
topic 0.035*"polic" + 0.019*"opposit" + 0.018*"work" + 0.017*"call" + 0.017*"probe" + 0.016*"talk" + 0.016*"govt" + 0.013*"blaze" + 0.010*"threat" + 0.010*"push"

score:0.02000470831990242	 
topic 0.022*"minist" + 0.019*"closer" + 0.018*"claim" + 0.017*"protest" + 0.013*"strike" + 0.012*"reject" + 0.011*"australia" + 0.010*"worker" + 0.010*"damag" + 0.009*"storm"

score:0.020004503428936005	 
topic 0.019*"record" + 0.016*"coast" + 0.016*"law" + 0.016*"market" + 0.016*"busi" + 0.015*"time" + 0.014*"gold" + 0.012*"trade" + 0.012*"guilti" + 0.011*"drive"

score:0.0200039483606815

In [45]:
#testing mdel on unseen document

In [46]:
unseen_document='My favourite sports are running and playing cricket'

In [47]:
#data preprocessing step for unseen document
bow_vector=dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup : -1*tup[1]):
    print('score {} \t Topic:{}'. format(score, lda_model.print_topic(index,5)))

score 0.56449294090271 	 Topic:0.022*"minist" + 0.019*"closer" + 0.018*"claim" + 0.017*"protest" + 0.013*"strike"
score 0.17790427803993225 	 Topic:0.019*"record" + 0.016*"coast" + 0.016*"law" + 0.016*"market" + 0.016*"busi"
score 0.1408858448266983 	 Topic:0.023*"nation" + 0.018*"win" + 0.016*"rule" + 0.010*"final" + 0.010*"blue"
score 0.01668582484126091 	 Topic:0.024*"miss" + 0.018*"lead" + 0.016*"search" + 0.015*"open" + 0.013*"aussi"
score 0.01667320355772972 	 Topic:0.023*"hospit" + 0.017*"report" + 0.016*"deal" + 0.013*"action" + 0.011*"find"
score 0.016672460362315178 	 Topic:0.024*"polic" + 0.019*"jail" + 0.016*"arrest" + 0.014*"break" + 0.012*"land"
score 0.016672244295477867 	 Topic:0.047*"council" + 0.043*"plan" + 0.034*"charg" + 0.030*"court" + 0.029*"face"
score 0.01667158491909504 	 Topic:0.035*"kill" + 0.030*"crash" + 0.027*"death" + 0.022*"polic" + 0.018*"road"
score 0.016671322286128998 	 Topic:0.035*"polic" + 0.019*"opposit" + 0.018*"work" + 0.017*"call" + 0.017*"pro